In [1]:

import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
DATASET_DIR = './data/'
GLOVE_DIR = './glove.6B/'
SAVE_DIR = './'

X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1', 'domain1_score', 'essay_id'])
prompt = X['essay_set']
# X = X.drop(columns=['essay_set'])
y = open(os.path.join(DATASET_DIR, 'prompt.txt')).read().splitlines()
print(len(y))
X.shape

8


(12976, 2)

In [2]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    # featureVec = np.zeros((num_features,),dtype="float32")
    # num_words = 0.
    # index2word_set = set(model.wv.index_to_key)
    # for word in words:
    #     if word in index2word_set:
    #         num_words += 1
    #         featureVec = np.add(featureVec,model.wv[word])        
    # featureVec = np.divide(featureVec,num_words)
    # return featureVec
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0
    
    for word in words:
        if word in model:
            nwords += 1
            featureVec = np.add(featureVec, model[word])
            
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [3]:
sentences = []
for i in y:
  sentences += essay_to_sentences(i, remove_stopwords = True)

for essay in X['essay']:
  sentences += essay_to_sentences(essay, remove_stopwords = True)


In [3]:
from gensim.models import Word2Vec
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

# print("Training Word2Vec Model...")
# model = Word2Vec(sentences, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, sample = downsampling)

# model.init_sims(replace=True)
# model.save('./w2v_otd.model')

# model = Word2Vec.load("./mysite/grader/deep_learning_files/word2vecmodel.model")
import gensim.downloader as api
model = api.load("word2vec-google-news-300")


In [4]:
# clean_essays = []
# for essay_v in X['essay']:
#     clean_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
# data_vecs = getAvgFeatureVecs(clean_essays, model, num_features)
# # save it to csv
# df = pd.DataFrame(data_vecs)
# df.to_csv(os.path.join(SAVE_DIR, 'data_vecs.csv'), index=False)

# load datavecs
data_vecs = pd.read_csv(os.path.join(SAVE_DIR, 'data_vecs.csv'))

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_z_scores(cosine_similarities, mean, std_dev):
    """Calculate the z-scores based on cosine similarities."""
    return (cosine_similarities - mean) / std_dev 

In [6]:
from sklearn.model_selection import train_test_split
import joblib

X_train, X_valid, y_train, y_valid= train_test_split(data_vecs, prompt, test_size=0.1, random_state=42)
y_valid = np.array(y_valid)
# standardize
# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# X_train = ss.fit_transform(X_train)
# X_valid = ss.transform(X_valid)
# joblib.dump(ss, os.path.join(SAVE_DIR, 'ss.pkl'))

# shuffle y_valid to create off topic records
true = y_valid.copy()
np.random.seed(42)
np.random.shuffle(y_valid)

true = [1 if true[i] == y_valid[i] else 0 for i in range(y_valid.shape[0])]

In [7]:

# y_sentences = []
# for i in y:
#   y_sentences.append(essay_to_wordlist(i, remove_stopwords = True))
# prompt_vector = getAvgFeatureVecs(y_sentences, model, num_features)
# # save it for later use in csv
# prompt_vector = pd.DataFrame(prompt_vector)
# prompt_vector.to_csv(os.path.join(SAVE_DIR, 'prompt_vector.csv'), index=False)

prompt_vector = np.array(pd.read_csv(os.path.join(SAVE_DIR, 'prompt_vector.csv')))

In [ ]:
# embed prompt sentences

train_essays = data_vecs

# cos similarities between each training essays with the training set
cosine_similarities1 = cosine_similarity(X_train, X_train)
max_cos_sim1 = np.max(cosine_similarities1, axis = 1)
# cos similarity between training essays and its corresponding prompt
prompt_similarities = np.diag(cosine_similarity(X_train, prompt_vector[y_train - 1]))

# z score parameters
mean_max_cos_sim = np.mean(max_cos_sim1)
std_dev_max_cos_sim = np.std(max_cos_sim1)

mean_prompt_cos = np.mean(prompt_similarities)
std_dev_prompt_cos = np.std(prompt_similarities)

def modelA(novel_essays, novel_essays_prompt):
  '''
  predict off-topicity using cosine similarity and z-scores
  '''
  # cos similarity between novel essay and training essays
  cosine_similarities = cosine_similarity(novel_essays, X_train)
  max_cos_sim = np.max(cosine_similarities, axis = 1)

  # cos similarity between novel essay and its corresponding prompt
  prompt_cos = np.diag(cosine_similarity(novel_essays, novel_essays_prompt))

  z_scores_max_cos_sim = calculate_z_scores(max_cos_sim, mean_max_cos_sim, std_dev_max_cos_sim)
  z_scores_prompt_cos = calculate_z_scores(prompt_cos, mean_prompt_cos, std_dev_prompt_cos)

  return z_scores_max_cos_sim, z_scores_prompt_cos

In [ ]:
# stores mean and std_dev
# cos similarities between each training essays with the training set

X_train, X_valid, y_train, y_valid= train_test_split(data_vecs, prompt, test_size=0.05, random_state=42)
y_valid = np.array(y_valid)

true = y_valid.copy()
np.random.seed(42)
np.random.shuffle(y_valid)

true = [1 if true[i] == y_valid[i] else 0 for i in range(y_valid.shape[0])]

max_cos_sim1 = np.max(cosine_similarity(X_train, X_train), axis = 1)
# cos similarity between training essays and its corresponding prompt
prompt_similarities = np.diag(cosine_similarity(X_train, prompt_vector[y_train - 1]))

# z score parameters
mean_max_cos_sim = np.mean(max_cos_sim1)
std_dev_max_cos_sim = np.std(max_cos_sim1)
# mean_max_cos_sim = np.median(max_cos_sim1)
# std_dev_max_cos_sim = (np.percentile(max_cos_sim1, 75) - np.percentile(max_cos_sim1, 25)) / 1.349

mean_prompt_cos = np.mean(prompt_similarities)
std_dev_prompt_cos = np.std(prompt_similarities)
# mean_prompt_cos = np.median(prompt_similarities)
# std_dev_prompt_cos = (np.percentile(prompt_similarities, 75) - np.percentile(prompt_similarities, 25)) / 1.349

max_cos_sim = np.max(cosine_similarity(X_valid, X_train), axis = 1)

# cos similarity between novel essay and its corresponding prompt
prompt_cos = np.diag(cosine_similarity(X_valid, prompt_vector[y_valid - 1]))

z_scores = []
z_scores.append(calculate_z_scores(max_cos_sim, mean_max_cos_sim, std_dev_max_cos_sim))
z_scores.append(calculate_z_scores(prompt_cos, mean_prompt_cos, std_dev_prompt_cos))


from sklearn.metrics import roc_curve
fpr, tpr, threshold = roc_curve(true, z_scores[0])
fpr1, tpr1, threshold1 = roc_curve(true, z_scores[1])
opt_threshold1 = threshold1[np.argmax(tpr1 - fpr1)]
opt_threshold = threshold[np.argmax(tpr - fpr)]
print(opt_threshold, opt_threshold1)
# accuracy, confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix
pred = [1 if z_scores[0][i] > opt_threshold and z_scores[1][i] > opt_threshold1 else 0 for i in range(len(z_scores[0]))]
print(accuracy_score(true, pred))
print(confusion_matrix(true, pred))

# stores mean and std_dev in a csv file
precompute = pd.DataFrame({'mean_max_cos_sim': [mean_max_cos_sim], 'std_dev_max_cos_sim': [std_dev_max_cos_sim], 'mean_prompt_cos': [mean_prompt_cos], 'std_dev_prompt_cos': [std_dev_prompt_cos]})
precompute.to_csv(os.path.join(SAVE_DIR, 'precompute.csv'), index=False)

: 

In [ ]:
z_scores= modelA(X_valid, prompt_vector[y_valid - 1])
print(z_scores[0].shape, z_scores[1].shape)

(649,) (649,)


In [ ]:
# confusion matrix, roc curve, accuracy
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, roc_curve, auc, RocCurveDisplay
import matplotlib.pyplot as plt

fpr, tpr, threshold = roc_curve(true, z_scores[0])
fpr1, tpr1, threshold1 = roc_curve(true, z_scores[1])
# find threshold
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = threshold[optimal_idx]
print(optimal_threshold)
optimal_idx1 = np.argmax(tpr1 - fpr1)
optimal_threshold1 = threshold1[optimal_idx1]
print(optimal_threshold1)

-28524317292836.273
-0.08908073589062888


In [ ]:
# confusion matrix, accuracy
pred = [1 if i >= optimal_threshold and j >= optimal_threshold1 else 0 for i, j in zip(z_scores[0], z_scores[1])]
print(confusion_matrix(true, pred))
print(accuracy_score(true, pred))

[[561   1]
 [ 74  13]]
0.884437596302003


In [ ]:
# threshold using percentile
percentile = 85
threshold2 = np.percentile(z_scores[0], percentile)
threshold3 = np.percentile(z_scores[1], percentile)
print(threshold2, threshold3)

# confusion matrix, accuracy
pred = [1 if i >= threshold2 and j >= threshold3 else 0 for i, j in zip(z_scores[0], z_scores[1])]
print(confusion_matrix(true, pred))
print(accuracy_score(true, pred))

-36593194584319.57 0.47074779756973545
[[556   6]
 [ 75  12]]
0.8751926040061633
